#### This notebook was created and run in Google Colab

In [1]:
!pip install transformers

     |████████████████████████████████| 675kB 3.4MB/s 
     |████████████████████████████████| 1.1MB 17.7MB/s 
     |████████████████████████████████| 3.8MB 21.7MB/s 
     |████████████████████████████████| 890kB 48.4MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893260 sha256=5bbd6025c45aa5828544a0cfd94ec5154f41d39e32fefd50055255288d5a6a69
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


In [2]:
import tensorflow as tf
import transformers
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import tensorflow as tf
import transformers
import tokenizers
from sklearn.model_selection import StratifiedKFold
import tensorflow.keras.backend as K
from keras.utils.np_utils import to_categorical  
import shutil 

Using TensorFlow backend.


In [3]:
try:
# TPU detection.
  tpu = tf.distribute.cluster_resolver.TPUClusterResolver()
  print('Running on TPU ', tpu.master())
except ValueError:
  tpu = None

if tpu:
  tf.config.experimental_connect_to_cluster(tpu)
  tf.tpu.experimental.initialize_tpu_system(tpu)
  strategy = tf.distribute.experimental.TPUStrategy(tpu)
  AUTO = tf.data.experimental.AUTOTUNE
else:
# Default distribution strategy in Tensorflow. Works on CPU and single GPU.
  strategy = tf.distribute.get_strategy()

print("REPLICAS: ", strategy.num_replicas_in_sync)

INFO:absl:Entering into master device scope: /job:worker/replica:0/task:0/device:CPU:0


Running on TPU  grpc://10.79.241.82:8470
INFO:tensorflow:Initializing the TPU system: grpc://10.79.241.82:8470


INFO:tensorflow:Initializing the TPU system: grpc://10.79.241.82:8470


INFO:tensorflow:Clearing out eager caches


INFO:tensorflow:Clearing out eager caches


...

INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


REPLICAS:  8


In [4]:
train = pd.read_csv("https://www.dropbox.com/s/w6s6hszo1amf53q/train.csv?dl=1")
test = pd.read_csv("https://www.dropbox.com/s/ett5vs1tz6xl0w0/test.csv?dl=1")
sample_sub = pd.read_csv("https://www.dropbox.com/s/8da3kbzcseumd2p/sample_submission.csv?dl=1")

In [5]:
LEARNING_RATE = 3e-4
BATCH_SIZE = 16 * strategy.num_replicas_in_sync

In [6]:
train.head()

,textID,text,selected_text,sentiment
0,cb774db0d1,"I`d have responded, if I were going","I`d have responded, if I were going",neutral
1,549e992a42,Sooo SAD I will miss you here in San Diego!!!,Sooo SAD,negative
2,088c60f138,my boss is bullying me...,bullying me,negative
3,9642c003ef,what interview! leave me alone,leave me alone,negative
4,358bd9e861,"Sons of ****, why couldn`t they put them on t...","Sons of ****,",negative


In [7]:
test.head()

,textID,text,sentiment
0,f87dea47db,Last session of the day http://twitpic.com/67ezh,neutral
1,96d74cb729,Shanghai is also really exciting (precisely -...,positive
2,eee518ae67,"Recession hit Veronique Branquinho, she has to...",negative
3,01082688c6,happy bday!,positive
4,33987a8ee5,http://twitpic.com/4w75p - I like it!!,positive


In [8]:
#print(train.dtypes)
train['textID'] = train['textID'].apply(str)
train['text'] = train['text'].apply(str)
train['selected_text'] = train['selected_text'].apply(str)
train['sentiment'] = train['sentiment'].apply(str)

test['textID'] = test['textID'].apply(str)
test['text'] = test['text'].apply(str)
test['sentiment'] = test['sentiment'].apply(str)

In [9]:
ex = train.loc[3,:]
context = ex.text
answer = ex.selected_text
print(context)
print(answer)

 what interview! leave me alone
leave me alone


In [10]:
# Download RoBERTa vocabulary file from my Gdrive
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [11]:
try:
  shutil.copytree(r'/content/gdrive/My Drive/Colab_data/tf-roberta/', r'tf-roberta')
except FileExistsError:
  pass

In [12]:
train.sentiment.value_counts()

neutral     11118
positive     8582
negative     7781
Name: sentiment, dtype: int64

In [13]:
train.loc[train.sentiment == 'neutral',['text', 'selected_text']].apply(lambda row: len(row['text'].strip())-
                                                                        len(row['selected_text'].strip()), axis=1 ).value_counts()

0      10005
1        171
2        116
3         66
27        57
       ...  
91         1
59         1
106        1
90         1
99         1
Length: 98, dtype: int64

In [14]:
ROBERTA_PATH = './tf-roberta/'
# Intialize the RoBERTa base tokenizer
tokenizer = tokenizers.ByteLevelBPETokenizer(vocab_file = ROBERTA_PATH+'vocab-roberta-base.json', 
                                             merges_file = ROBERTA_PATH+'merges-roberta-base.txt',
                                            lowercase = True,add_prefix_space=True)

def preprocess_input_data_roberta(question, context, answer, tokenizer, max_len):
    
# Tokenize and encode the question (sentiment) and the context (tweet) with special tokens
    context = " " + " ".join(context.split()) # This is done because some tweets start with double spaces, the tokenization then gets messed up
    answer = " " + " ".join(answer.split())
    enc_question = tokenizer.encode(question)
    enc_context = tokenizer.encode(context)
    input_ids = [0] + enc_question.ids + [2,2] +   enc_context.ids + [2]
    input_tokens = enc_question.tokens + enc_context.tokens
    token_type_ids = [0]* len(input_ids) # not relevant for RoBERTa
    attention_mask = [1]* len(input_ids)
    
    offsets_question = enc_question.offsets
    offsets_context = enc_context.offsets
    offsets = [(0,0)] + offsets_question + [(0,0)]*2 + offsets_context +[(0,0)]
    
    target_char_start  = context.find(answer)
    target_char_end = target_char_start + len(answer) - 1
    char_targets = [0]*len(context)
    for i in range(target_char_start,target_char_end+1):
        char_targets[i] = 1
    targets_index_context = []    
    for ind, (i,j) in enumerate(offsets_context):
        if sum(char_targets[i:j]) > 0:
            targets_index_context.append(ind) 
           
    target_start_ind = targets_index_context[0] + 4 
    target_end_ind   = targets_index_context[-1] + 4
    

        # padding -- pad the vectors if their lengths exceed max_len
    pad_len = max_len - len(token_type_ids)
    if(pad_len> 0):
        token_type_ids = token_type_ids + [0]*pad_len
        input_ids = input_ids + [1]*pad_len    # [1] is the <pad> token in RoBERTa
        attention_mask = attention_mask + [0]*pad_len
        offsets = offsets + [(0,0)]*pad_len    

        
    output_dict = {'token_type_ids': token_type_ids,
                  'input_ids': input_ids,
                  'token_type_ids': token_type_ids,
                  'target_start': target_start_ind,
                  'target_end': target_end_ind,
                  'input_tokens': input_tokens,
                   'offsets': offsets,
                   'attention_mask': attention_mask,
                   'sentiment': question,
                   'context': context,
                   'answer': answer
                  }
    return output_dict

In [15]:
data_example = preprocess_input_data_roberta(train.sentiment[3], train.text[3], train.selected_text[3], tokenizer, max_len = 15)
print('Sentiment:{}'.format(data_example['sentiment']))
print('Tweet:{}'.format(data_example['context']))
print('Selected text:{}'.format(data_example['answer']))
print('Input tokens: {}'.format(data_example['input_tokens']))
print('Token type ids: {}'.format(data_example['token_type_ids']))
print('Input_ids: {}'.format(data_example['input_ids']))
print('Target start index: {}, Target end index: {}'.format(data_example['target_start'], data_example['target_end']))
print('Attention mask: {}'.format(data_example['attention_mask']))
print('offsets: {}'.format(data_example['offsets']))

Sentiment:negative
Tweet: what interview! leave me alone
Selected text: leave me alone
Input tokens: ['Ġnegative', 'Ġwhat', 'Ġinterview', '!', 'Ġleave', 'Ġme', 'Ġalone']
Token type ids: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
Input_ids: [0, 2430, 2, 2, 99, 1194, 328, 989, 162, 1937, 2, 1, 1, 1, 1]
Target start index: 7, Target end index: 9
Attention mask: [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0]
offsets: [(0, 0), (0, 8), (0, 0), (0, 0), (0, 5), (5, 15), (15, 16), (16, 22), (22, 25), (25, 31), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0)]


In [16]:
# Tokenize and encode the training set

max_len = 128

input_ids = np.ones((train.shape[0],max_len), dtype = np.int32)
token_type_ids = np.zeros((train.shape[0],max_len), dtype = np.int32)
attention_mask = np.zeros((train.shape[0],max_len), dtype = np.int32)
start_ids = np.zeros((train.shape[0],max_len), dtype = np.int32)
end_ids = np.zeros((train.shape[0],max_len), dtype = np.int32)


for i in range(train.shape[0]):
    question = train.sentiment[i]
    context= train.text[i]
    answer = train.selected_text[i]
    processed_data = preprocess_input_data_roberta(question, context, answer, tokenizer, max_len = max_len)
    input_ids[i,:] = processed_data['input_ids']
    token_type_ids[i,:] = processed_data['token_type_ids']
    attention_mask[i,:] = processed_data['attention_mask']
    start_ids[i,:] = to_categorical(processed_data['target_start'], num_classes = max_len)
    end_ids[i,:] = to_categorical(processed_data['target_end'], num_classes = max_len)

In [17]:
# Tokenize and encode the test set
def preprocess_test_data_roberta(question, context, tokenizer, max_len):
    
# Tokenize and encode the question (sentiment) and the context (tweet) with special tokens
    context = " " + " ".join(context.split())
    enc_question = tokenizer.encode(question)
    enc_context = tokenizer.encode(context)
    input_ids = [0] + enc_question.ids + [2,2] +   enc_context.ids + [2]
    input_tokens = enc_question.tokens + enc_context.tokens
    attention_mask = [1]* len(input_ids)
        
    # padding -- pad the vectors if their lengths exceed max_len, else truncate at max_len
    pad_len = max_len - len(input_ids)
    if(pad_len> 0):
        input_ids = input_ids + [1]*pad_len
        attention_mask = attention_mask + [0]*pad_len

    
    output_dict = {'input_ids': input_ids,
                  'input_tokens': input_tokens,
                   'attention_mask': attention_mask,
                   'sentiment': question,
                   'context': context,
                  }
    return output_dict

input_ids_test = np.ones((test.shape[0],max_len), dtype = np.int32)
token_type_ids_test = np.zeros((test.shape[0],max_len), dtype = np.int32)
attention_mask_test = np.zeros((test.shape[0],max_len), dtype = np.int32)


for i in range(test.shape[0]):
    question = test.sentiment[i]
    context= test.text[i]
    processed_data_test = preprocess_test_data_roberta(question, context, tokenizer, max_len = max_len)
    input_ids_test[i,:] = processed_data_test['input_ids']
    attention_mask_test[i,:] = processed_data_test['attention_mask']



In [18]:


def build_model():
    ids = tf.keras.layers.Input((max_len,), dtype=tf.int32)
    tok = tf.keras.layers.Input((max_len,), dtype=tf.int32)
    att = tf.keras.layers.Input((max_len,), dtype=tf.int32)
    
    config = transformers.RobertaConfig.from_pretrained(ROBERTA_PATH+'config-roberta-base.json', output_hidden_states=True)  
    roberta_tf = transformers.TFRobertaModel.from_pretrained('roberta-base', config=config)
    x = roberta_tf({'input_ids': ids, 'token_type_ids': tok, 'attention_mask': att})[0]

 # Refer to https://web.stanford.edu/class/archive/cs/cs224n/cs224n.1194/reports/default/15812785.pdf   


#    print(x.shape)
    
    h1 = tf.keras.layers.Dropout(0.2)(x) 
    h1 = tf.keras.layers.Dense(1)(h1)
    h1 = tf.keras.layers.Flatten()(h1)
    h1 = tf.keras.layers.Activation('softmax', name='start_token_id')(h1)

    h2 = tf.keras.layers.Dropout(0.2)(x) 
    h2 = tf.keras.layers.Dense(1)(h2)
    h2 = tf.keras.layers.Flatten()(h2)
    h2 = tf.keras.layers.Activation('softmax', name='end_token_id')(h2)

    model = tf.keras.models.Model(inputs=[ids, att, tok], outputs=[h1,h2])
    optimizer = tf.keras.optimizers.Adam(learning_rate=LEARNING_RATE)
    model.compile(optimizer = optimizer, loss='categorical_crossentropy')

    return model


In [19]:
model = build_model()
model.summary()


Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            [(None, 128)]        0                                            
__________________________________________________________________________________________________
input_1 (InputLayer)            [(None, 128)]        0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            [(None, 128)]        0                                            
__________________________________________________________________________________________________
tf_roberta_model (TFRobertaMode ((None, 128, 768), ( 124645632   input_3[0][0]                    
                                                                 input_1[0][0]               

In [20]:
#    config = transformers.BertConfig()

def build_model_CNN():
    ids = tf.keras.layers.Input((max_len,), dtype=tf.int32)
    tok = tf.keras.layers.Input((max_len,), dtype=tf.int32)
    att = tf.keras.layers.Input((max_len,), dtype=tf.int32) 

    config = transformers.RobertaConfig.from_pretrained(ROBERTA_PATH+'config-roberta-base.json', output_hidden_states=True)  
    roberta_tf = transformers.TFRobertaModel.from_pretrained('roberta-base', config=config)
    x = roberta_tf({'input_ids': ids, 'token_type_ids': tok, 'attention_mask': att})[0]
 

#    print(x.shape)
    h1 = tf.keras.layers.Dropout(0.1)(x) 
    h1 = tf.keras.layers.Conv1D(128, 2,padding='same')(h1)
    h1 = tf.keras.layers.BatchNormalization()(h1)
    h1 = tf.keras.layers.ReLU()(h1)
    h1 = tf.keras.layers.Dense(1)(h1)
#    print(h1.shape)
    h1 = tf.keras.layers.Flatten()(h1)
#    print(h1.shape)
    h1 = tf.keras.layers.Activation('softmax', name='start_token_id')(h1)
#    print(h1.shape)

    h2 = tf.keras.layers.Dropout(0.1)(x) 
    h2 = tf.keras.layers.Conv1D(128, 2,padding='same')(h2)
    h2 = tf.keras.layers.BatchNormalization()(h2)
    h2 = tf.keras.layers.ReLU()(h2)
    h2 = tf.keras.layers.Dense(1)(h2)
    h2 = tf.keras.layers.Flatten()(h2)
    h2 = tf.keras.layers.Activation('softmax', name='end_token_id')(h2)

    model = tf.keras.models.Model(inputs=[ids, att, tok], outputs=[h1,h2])
    optimizer = tf.keras.optimizers.Adam(learning_rate=1e-4)
    model.compile(optimizer = optimizer, loss='categorical_crossentropy')

    return model


In [21]:

def build_CNN(num_filters=3,filter_sizes = (3,5)):
    ids = tf.keras.layers.Input((max_len,), dtype=tf.int32)
    tok = tf.keras.layers.Input((max_len,), dtype=tf.int32)
    att = tf.keras.layers.Input((max_len,), dtype=tf.int32)
    
    hidden_dims = 128
    
    config = transformers.RobertaConfig.from_pretrained(ROBERTA_PATH+'config-roberta-base.json')  
    roberta_tf = transformers.TFRobertaModel.from_pretrained('roberta-base', config=config)
    x = roberta_tf({'input_ids': ids, 'token_type_ids': tok, 'attention_mask': att})[0]

#    print(x.shape)
    h1 = tf.keras.layers.Dropout(0.1)(x) 
        # Convolutional block
    conv_blocks = []
    for sz in filter_sizes:
        conv = tf.keras.layers.Conv1D(filters=num_filters,
                             kernel_size=sz,
                             padding="same",
                             activation="relu",
                             strides=1)(h1)
        conv = tf.keras.layers.MaxPooling1D(pool_size=2)(conv)
        conv = tf.keras.layers.Flatten()(conv)
        conv_blocks.append(conv)
    h1 = tf.keras.layers.Concatenate()(conv_blocks) if len(conv_blocks) > 1 else conv_blocks[0]

    h1 = tf.keras.layers.Dropout(0.1)(h1)
    h1 = tf.keras.layers.Dense(hidden_dims)(h1)
    h1 = tf.keras.layers.Flatten()(h1)
    h1 = tf.keras.layers.Activation('softmax', name='start_token_id')(h1)



    h2 = tf.keras.layers.Dropout(0.1)(x) 
    conv_blocks = []
    for sz in filter_sizes:
        conv = tf.keras.layers.Conv1D(filters=num_filters,
                             kernel_size=sz,
                             padding="same",
                             activation="relu",
                             strides=1)(h2)
        conv = tf.keras.layers.MaxPooling1D(pool_size=2)(conv)
        conv = tf.keras.layers.Flatten()(conv)
        conv_blocks.append(conv)
    h2 = tf.keras.layers.Concatenate()(conv_blocks) if len(conv_blocks) > 1 else conv_blocks[0]

    h2 = tf.keras.layers.Dropout(0.1)(h2)
    h2 = tf.keras.layers.Dense(hidden_dims)(h2)
    h2 = tf.keras.layers.Flatten()(h2)
    h2 = tf.keras.layers.Activation('softmax', name='end_token_id')(h2)

    model = tf.keras.models.Model(inputs=[ids, att, tok], outputs=[h1,h2])
    optimizer = tf.keras.optimizers.Adam(learning_rate= 3e-5)
    model.compile(optimizer = optimizer, loss='categorical_crossentropy')

    return model

In [22]:
model = build_model_CNN()
model.summary()

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_6 (InputLayer)            [(None, 128)]        0                                            
__________________________________________________________________________________________________
input_4 (InputLayer)            [(None, 128)]        0                                            
__________________________________________________________________________________________________
input_5 (InputLayer)            [(None, 128)]        0                                            
__________________________________________________________________________________________________
tf_roberta_model_1 (TFRobertaMo ((None, 128, 768), ( 124645632   input_6[0][0]                    
                                                                 input_4[0][0]              

In [23]:
def jaccard(str1, str2): 
    a = set(str1.lower().split()) 
    b = set(str2.lower().split())
    c = a.intersection(b)
    return float(len(c)) / (len(a) + len(b) - len(c))

In [26]:
def scheduler(epoch):
    return LEARNING_RATE * 0.2**epoch


def train_model(input_ids, attention_mask, token_type_ids, input_ids_test,attention_mask_test,token_type_ids_test,kfold_n_splits = 5, epochs=3):
  jaccard_scores = []
  oof_start = np.zeros((input_ids.shape[0],max_len))
  oof_end = np.zeros((input_ids.shape[0],max_len))
  preds_start = np.zeros((input_ids_test.shape[0],max_len))
  preds_end = np.zeros((input_ids_test.shape[0],max_len))

  

  skf = StratifiedKFold(n_splits=kfold_n_splits,shuffle=True,random_state=12345)
  for fold,(idxT,idxV) in enumerate(skf.split(input_ids,train.sentiment.values)):
    K.clear_session()
    tf.tpu.experimental.initialize_tpu_system(tpu)
    
    print('#'*25)
    print('### FOLD {}'.format(fold+1))
    print('#'*25)
    X_train = (
        tf.data.Dataset
            .from_tensor_slices( ({'input_1':input_ids[idxT,],
                  'input_2':attention_mask[idxT,], 'input_3':token_type_ids[idxT,]},
                 {'start_token_id':start_ids[idxT,], 'end_token_id':end_ids[idxT,]}) )
            .shuffle(buffer_size = input_ids.shape[0])
            .repeat(epochs)
            .batch(BATCH_SIZE,drop_remainder=True)
            .prefetch(AUTO)
    )
     
    X_val = (
        tf.data.Dataset
            .from_tensor_slices( ({'input_1':input_ids[idxV,],
                  'input_2':attention_mask[idxV,], 'input_3':token_type_ids[idxV,]},
                 {'start_token_id':start_ids[idxV,], 'end_token_id':end_ids[idxV,]}) )
            .batch(4*BATCH_SIZE)
            .prefetch(AUTO)
    )


    X_test = (
        tf.data.Dataset
            .from_tensor_slices( ({'input_1':input_ids_test,
                  'input_2':attention_mask_test, 'input_3':token_type_ids_test},
                 ) )
            .batch(4*BATCH_SIZE)
            .prefetch(AUTO)
    )
    
    with strategy.scope():
      model = build_model_CNN()
        
   
        
    sv = tf.keras.callbacks.ModelCheckpoint(
        "roberta-fold{}.h5".format(fold+1), monitor='val_loss', verbose=1, save_best_only=True,
        save_weights_only=True, mode='auto', save_freq='epoch')
    
#    lr_scheduler = tf.keras.callbacks.LearningRateScheduler(scheduler)

    steps_per_epoch= int(np.ceil(input_ids[idxT,].shape[0]/BATCH_SIZE ))
    validation_steps = int(np.ceil(input_ids[idxV,].shape[0]/BATCH_SIZE ))


    model.fit(X_train, validation_data = X_val,
              epochs=epochs, verbose=1, callbacks=[sv])
    
    print('Loading model...')
    model.load_weights('roberta-fold{}.h5'.format(fold+1))
    shutil.copy2(r'roberta-fold{}.h5'.format(fold+1), r'/content/gdrive/My Drive/')

    print('Predicting out-of-fold answer span')
    oof_start[idxV,],oof_end[idxV,] = model.predict(X_val, verbose=1)
    # DISPLAY FOLD JACCARD
    jac_oof = []
    for k in idxV:
        a = np.argmax(oof_start[k,])
        b = np.argmax(oof_end[k,])
        true_answer =  " " + " ".join(train.loc[k,'selected_text'].split())
        context =  " " + " ".join(train.loc[k,'text'].split())
        if (train.loc[k, 'sentiment'] == 'neutral'):
              pred_answer = " " + " ".join(train.loc[k,'text'].split())
        elif a>b: 
            enc_question = tokenizer.encode(train.loc[k,'sentiment'])
            enc_context = tokenizer.encode(context)
            enc_ids = [0] + enc_question.ids + [2,2] +   enc_context.ids + [2]
            pred_answer = tokenizer.decode(enc_ids[a:]) 
        else:
            enc_question = tokenizer.encode(train.loc[k,'sentiment'])
            enc_context = tokenizer.encode(context)
            enc_ids = [0] + enc_question.ids + [2,2] +   enc_context.ids + [2]
            pred_answer = tokenizer.decode(enc_ids[a:b+1])       
        jac_oof.append(jaccard(pred_answer, true_answer))
    jaccard_scores.append(np.mean(jac_oof))
    print('FOLD %i Jaccard ='%(fold+1),np.mean(jac_oof))

    print('Predicting Test answer span')
    preds = model.predict(X_test, verbose=1)
    preds_start += preds[0]/kfold_n_splits
    preds_end += preds[1]/kfold_n_splits
  return {'pred_test_start': preds_start, 'pred_test_end': preds_end,
          'jaccard_kfold': jaccard_scores}


In [27]:
dataset = tf.data.Dataset.from_tensor_slices([1, 2, 3,4,5,6]).shuffle(buffer_size = 5).repeat(3).batch(3)
for element in dataset.as_numpy_iterator():
  print(element)

print('#'*25)
dataset = tf.data.Dataset.from_tensor_slices([1, 2, 3,4,5,6]).repeat(3).shuffle(buffer_size = 5).batch(3)
for element in dataset.as_numpy_iterator():
  print(element)

[1 2 6]
[3 5 4]
[3 5 6]
[2 4 1]
[5 4 3]
[1 2 6]
#########################
[2 3 5]
[4 2 4]
[1 6 1]
[6 3 1]
[5 6 5]
[4 2 3]


In [28]:
out = train_model(input_ids, attention_mask, token_type_ids, input_ids_test,attention_mask_test,token_type_ids_test,kfold_n_splits = 5, epochs=3)

INFO:tensorflow:Initializing the TPU system: grpc://10.79.241.82:8470


INFO:tensorflow:Initializing the TPU system: grpc://10.79.241.82:8470


INFO:tensorflow:Clearing out eager caches


INFO:tensorflow:Clearing out eager caches


INFO:tensorflow:Finished initializing TPU system.


...

INFO:tensorflow:Clearing out eager caches


INFO:tensorflow:Finished initializing TPU system.


INFO:tensorflow:Finished initializing TPU system.


#########################
### FOLD 5
#########################
Epoch 1/3


515/515 [==============================] - ETA: 0s - loss: 1.6566 - start_token_id_loss: 0.8019 - end_token_id_loss: 0.8547
Epoch 00001: val_loss improved from inf to 1.55882, saving model to roberta-fold5.h5
515/515 [==============================] - 108s 210ms/step - loss: 1.6566 - start_token_id_loss: 0.8019 - end_token_id_loss: 0.8547 - val_loss: 1.5588 - val_start_token_id_loss: 0.7406 - val_end_token_id_loss: 0.8183
Epoch 2/3
515/515 [==============================] - ETA: 0s - loss: 1.0675 - start_token_id_loss: 0.5241 - end_token_id_loss: 0.5434
Epoch 00002: val_loss did not improve from 1.55882
515/515 [==============================] - 79s 154ms/step - loss: 1.0675 - start_token_id_loss: 0.5241 - end_token_id_loss: 0.5434 - val_loss: 1.8613 - val_start_token_id_loss: 0.8863 - val_end_token_id_loss: 0.9750
Epoch 3/3
515/515 [==============================] - ETA: 0s - loss: 0.6501 - start_token_id_loss: 0.3272 - end_token_id_loss: 0.3229
Epoch 00003: val_loss did not improve f

In [ ]:
# predict on the test set
preds_start, preds_end = out['pred_test_start'], out['pred_test_end']

for i in range(input_ids_test.shape[0]):
    a = np.argmax(preds_start[i,])
    b = np.argmax(preds_end[i,])
    context = " " + " ".join(test.loc[i, 'text'])
    enc_context = tokenizer.encode(context)
    if (test.loc[i, 'sentiment'] == 'neutral'):
        pred_answer = context
        test.loc[i,'selected_text'] = pred_answer.strip()
    elif a>b:
        pred_answer = tokenizer.decode(enc_context.ids[a-4:])
        test.loc[i,'selected_text'] = pred_answer.strip()

    else:
        pred_answer = tokenizer.decode(enc_context.ids[a-4:b+1])   
        test.loc[i,'selected_text'] = pred_answer.strip()


In [31]:
def inference(input_ids, attention_mask, token_type_ids, input_ids_test,attention_mask_test,token_type_ids_test,kfold_n_splits = 5, epochs=3):
  jaccard_scores = []
  oof_start = np.zeros((input_ids.shape[0],max_len))
  oof_end = np.zeros((input_ids.shape[0],max_len))
  preds_start = np.zeros((input_ids_test.shape[0],max_len))
  preds_end = np.zeros((input_ids_test.shape[0],max_len))

  

  skf = StratifiedKFold(n_splits=kfold_n_splits,shuffle=True,random_state=12345)
  for fold,(idxT,idxV) in enumerate(skf.split(input_ids,train.sentiment.values)):
    K.clear_session()
    tf.tpu.experimental.initialize_tpu_system(tpu)
    
    print('#'*25)
    print('### FOLD {}'.format(fold+1))
    print('#'*25)

    X_val = (
        tf.data.Dataset
            .from_tensor_slices( ({'input_1':input_ids[idxV,],
                  'input_2':attention_mask[idxV,], 'input_3':token_type_ids[idxV,]},
                 {'start_token_id':start_ids[idxV,], 'end_token_id':end_ids[idxV,]}) )
            .batch(4*BATCH_SIZE)
            .prefetch(AUTO)
    )
    X_test = (
        tf.data.Dataset
            .from_tensor_slices( ({'input_1':input_ids_test,
                  'input_2':attention_mask_test, 'input_3':token_type_ids_test},
                 ) )
            .batch(4*BATCH_SIZE)
            .prefetch(AUTO)
    )
    
    with strategy.scope():
        model = build_model_CNN()

    shutil.copy2(r'/content/gdrive/My Drive/roberta-fold{}.h5'.format(fold+1), r'.')
    print('Loading model...')
    model.load_weights('roberta-fold%i.h5'%(fold+1))
    

    print('Predicting out-of-fold answer span')
    oof_start[idxV,],oof_end[idxV,] = model.predict(X_val, verbose=1)
      
    # DISPLAY FOLD JACCARD
    jac_oof = []
    for k in idxV:
        a = np.argmax(oof_start[k,])
        b = np.argmax(oof_end[k,])
        true_answer =  " " + " ".join(train.loc[k,'selected_text'].split())
        context = " " + " ".join(train.loc[k,'text'].split())
        if (train.loc[k, 'sentiment'] == 'neutral'):
              pred_answer = " " + " ".join(train.loc[k,'text'].split())
        elif a>b: 
            enc_question = tokenizer.encode(train.loc[k,'sentiment'])
            enc_context = tokenizer.encode(context)
            enc_ids = [0] + enc_question.ids + [2,2] +   enc_context.ids + [2]
            pred_answer = tokenizer.decode(enc_ids[a:]) 
        else:
            enc_question = tokenizer.encode(train.loc[k,'sentiment'])
            enc_context = tokenizer.encode(context)
            enc_ids = [0] + enc_question.ids + [2,2] +   enc_context.ids + [2]
            pred_answer = tokenizer.decode(enc_ids[a:b+1])       
        jac_oof.append(jaccard(pred_answer, true_answer))
    jaccard_scores.append(np.mean(jac_oof))
    print('FOLD %i Jaccard ='%(fold+1),np.mean(jac_oof))

    print('Predicting Test answer span')
    preds = model.predict(X_test, verbose=1)
    preds_start += preds[0]/kfold_n_splits
    preds_end += preds[1]/kfold_n_splits
  return {'pred_test_start': preds_start, 'pred_test_end': preds_end,
          'jaccard_kfold': jaccard_scores}


In [32]:
out_inf = inference(input_ids, attention_mask, token_type_ids, input_ids_test,attention_mask_test,token_type_ids_test,kfold_n_splits = 5)

INFO:tensorflow:Initializing the TPU system: grpc://10.79.241.82:8470


INFO:tensorflow:Initializing the TPU system: grpc://10.79.241.82:8470


INFO:tensorflow:Clearing out eager caches


INFO:tensorflow:Clearing out eager caches


INFO:tensorflow:Finished initializing TPU system.


...

INFO:tensorflow:Clearing out eager caches


INFO:tensorflow:Clearing out eager caches


INFO:tensorflow:Finished initializing TPU system.


INFO:tensorflow:Finished initializing TPU system.


#########################
### FOLD 5
#########################
Loading model...
Predicting out-of-fold answer span
11/11 [==============================] - 13s 1s/step
FOLD 5 Jaccard = 0.6897554558548077
Predicting Test answer span
7/7 [==============================] - 7s 964ms/step


In [33]:
print(np.mean(out_inf['jaccard_kfold']))

0.7003879507669898
